In [1]:
!pip install tensorflow==2.3.1 gym keras-rl2 gym[atari]

In [2]:
import gym 
import os
import random

In [3]:
!python -m retro.import .\ROMS\ROMS

^C


In [4]:
env = gym.make('SpaceInvaders-v0')
height, width, channels = env.observation_space.shape
actions = env.action_space.n

In [5]:
env.unwrapped.get_action_meanings()


['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [6]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = random.choice([0,1,2,3,4,5])
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:110.0
Episode:2 Score:120.0
Episode:3 Score:155.0
Episode:4 Score:180.0
Episode:5 Score:105.0


In [7]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

Traceback (most recent call last):
  File "C:\Users\dymon\anaconda3\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\dymon\anaconda3\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\dymon\anaconda3\lib\site-packages\retro\import\__main__.py", line 4, in <module>
    main()
  File "C:\Users\dymon\anaconda3\lib\site-packages\retro\scripts\import_path.py", line 42, in main
    with open(filepath, "rb") as f:
FileNotFoundError: [Errno 2] No such file or directory: '.\\ROMS\\ROMS\\Pursuit of the Pink Panther (Pink Panther - The Video Game, Adventures of the Pink Panther) (1983) (Probe 2000 - NAP, Roger Booth, Todd Marshall, Robin McDaniel, Jim Wickstead) (3152VC) (Prototype) ~.bin'


Importing Adventure-Atari2600
Importing AirRaid-Atari2600
Importing Alien-Atari2600
Importing Amidar-Atari2600
Importing Assault-Atari2600
Importing Asterix-Atari2600
Importing Asterix-Atari2600
Importing Asteroids-Atari2600
Importing Atlantis-Atari2600
Importing BankHeist-Atari2600
Importing BattleZone-Atari2600
Importing BeamRider-Atari2600
Importing Berzerk-Atari2600
Importing Bowling-Atari2600
Importing Boxing-Atari2600
Importing Breakout-Atari2600
Importing Carnival-Atari2600
Importing Centipede-Atari2600
Importing ChopperCommand-Atari2600
Importing CrazyClimber-Atari2600
Importing Defender-Atari2600
Importing DemonAttack-Atari2600
Importing DoubleDunk-Atari2600
Importing ElevatorAction-Atari2600
Importing Enduro-Atari2600
Importing FishingDerby-Atari2600
Importing Freeway-Atari2600
Importing Frostbite-Atari2600
Importing Gopher-Atari2600
Importing Gravitar-Atari2600
Importing Hero-Atari2600
Importing IceHockey-Atari2600
Importing Jamesbond-Atari2600
Importing JourneyEscape-Atari2

In [8]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3,height, width, channels)))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Convolution2D(64, (3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [16]:
model = build_model(height, width, channels, actions)

In [17]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 3, 51, 39, 32)     6176      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 24, 18, 64)     32832     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 22, 16, 64)     36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 67584)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               34603520  
_________________________________________________________________
dense_4 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_5 (Dense)              (None, 6)                

In [15]:
del model

NameError: name 'model' is not defined

In [11]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [12]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg', 
                   nb_actions=actions, nb_steps_warmup=1000
                  )
    return dqn

In [18]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-4))

In [ ]:
dqn.fit(env, nb_steps=10000, visualize=False, verbose=2)

In [20]:
scores = dqn.test(env, nb_episodes=10, visualize=True)
print(np.mean(scores.history['episode_reward']))

Testing for 10 episodes ...
Episode 1: reward: 20.000, steps: 341
Episode 2: reward: 30.000, steps: 343
Episode 3: reward: 135.000, steps: 688
Episode 4: reward: 155.000, steps: 581
Episode 5: reward: 90.000, steps: 465
Episode 6: reward: 95.000, steps: 439
Episode 7: reward: 75.000, steps: 495
Episode 8: reward: 75.000, steps: 419
Episode 9: reward: 105.000, steps: 482
Episode 10: reward: 60.000, steps: 372
84.0
